# Analytical Verification: The Curie-Weiss Model

## Neil D. Lawrence

## November 2025

In this notebok we validate the energy-entropy equivalence theorem from the Inaccessible Game using using the Curie-Weiss model, a mean-field system where the energy-entropy equivalence can be tested exactly across phase transitions.


## The Curie-Weiss Model

The Curie-Weiss model describes $n$ interacting binary spins with ferromagnetic coupling
$$
E(\mathbf{x}) = -\frac{J}{2n} \left(\sum_{i=1}^n x_i\right)^2 - h \sum_{i=1}^n x_i,
$$
where $x_i \in \{-1, +1\}$, $J > 0$ is the coupling strength, and $h$ is an external field. In the thermodynamic limit $n \to \infty$, the system is characterised by its magnetisation $m = \langle x_i \rangle$, which satisfies the self-consistency equation,
$$
m = \tanh(\beta(Jm + h)).
$$
Conditioned on the magnetisation the spins become independent, in a naive Bayes manner. This renders the model analytic in its mean field approximation but ensures that it can still exhibit non-trivial correlations that transgress the assumptions made when characterising the regime where the entropy-energy theorem applies.

### Phase Transition

The model exhibits a phase transition at critical inverse temperature $\beta_c = 1/J$ (equivalent to critical temperature $T_c = J$).

* *Disordered phase* ($\beta < \beta_c$): $m = 0$ (paramagnetic)
* *Ordered phase* ($\beta > \beta_c$): $m \neq 0$ (ferromagnetic)

This phase transition provides an good test case for our theorem: the disordered phase should satisfy $\nabla_m I \approx 0$ (equivalence holds), while the ordered phase should exhibit $\nabla_m I \gg 0$ (equivalence fails).


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Set plot style
plt.style.use('seaborn-v0_8-whitegrid' if 'seaborn-v0_8-whitegrid' in plt.style.available else 'default')


In [ ]:
plt.rcParams.update({
    'font.size': 14,
    'font.family': 'serif',
    'axes.labelsize': 18,
    'xtick.labelsize': 16,
    'ytick.labelsize': 16,
    'legend.fontsize': 'medium',
})

In [ ]:
import numpy as np
import curie_weiss_equivalence as cw

In [ ]:
# %load -s curie_weiss_magnetisation /Users/neil/lawrennd/the-inaccessible-game/curie_weiss_equivalence.py
def curie_weiss_magnetisation(beta, J, h, m_init=None):
    """
    Solve self-consistency equation: m = tanh(β(Jm + h))
    
    For h=0 and β > β_c = 1/J, there are three solutions:
    - m = 0 (unstable)
    - m = ±m* (stable, ferromagnetic)
    
    Parameters:
    -----------
    beta : float
        Inverse temperature
    J : float
        Coupling strength
    h : float
        External field
    m_init : float, optional
        Initial guess for magnetisation. If None, chooses based on phase.
        
    Returns:
    --------
    m : float
        Self-consistent magnetisation (stable solution)
    """
    def self_consistency(m):
        return m - np.tanh(beta * (J * m + h))
    
    beta_c = 1.0 / J  # Critical inverse temperature
    
    # Choose initial guess based on phase
    if m_init is None:
        if abs(h) > 0.01:
            # With field, start from field direction
            m_init = np.sign(h) * 0.5
        elif beta > beta_c:
            # Below T_c (ordered phase): start away from m=0 to find ferromagnetic solution
            m_init = 0.5  # Try to find positive magnetisation solution
        else:
            # Above T_c (disordered phase): m=0 is the only solution
            m_init = 0.0
    
    try:
        # Try to find solution with given initial guess
        m = fsolve(self_consistency, m_init, full_output=False)[0]
        
        # Check physical bounds
        if abs(m) > 1.0:
            return np.nan
        
        # Check stability: d(tanh(β(Jm+h)))/dm < 1
        # If unstable and below T_c, try to find stable solution
        stability = beta * J / (np.cosh(beta * (J * m + h)))**2
        if stability > 1.0 and abs(m) < 0.01 and beta > beta_c and abs(h) < 0.01:
            # m=0 is unstable, find ferromagnetic solution
            for init_guess in [0.3, 0.5, 0.7, -0.3, -0.5, -0.7]:
                m_try = fsolve(self_consistency, init_guess, full_output=False)[0]
                if abs(m_try) > 0.1 and abs(m_try) < 1.0:
                    # Check this solution is stable
                    stab_try = beta * J / (np.cosh(beta * (J * m_try + h)))**2
                    if stab_try < 1.0:
                        return abs(m_try)  # Return positive branch by convention
            # If still can't find stable solution, return what we have
            return abs(m)
        
        return m
    except:
        return np.nan


In [ ]:
J = 1.0   # Coupling strength
n = 1000   # System size (for exact computation, feasible up to ~1000)
h_sym = 0.01  # Small external field to break symmetry (for h=0 scans)

## Experiment 1: Critical Scaling Breakdown

The first experiment demonstrates the breakdown of energy-entropy equivalence across the phase transition.



## Critical Scaling Breakdown

Prediction 1: Angle between gradients scales non-trivially near T_c.

cos(θ) ≈ 1 - C|T - T_c|^α

At T_c = J (β_c = 1/J), equivalence should break down.


Plot the results

In [ ]:
beta_c = 1.0 / J  # Critical point
T_c = 1.0 / beta_c

# Scan temperatures around T_c
beta_range = np.linspace(0.5 * beta_c, 2.0 * beta_c, 100)
T_range = 1.0 / beta_range

angles = []
magnetisations = []
grad_I_values = []

print(f"Computing {len(beta_range)} temperatures with exact gradients (n={n})...")
for i, beta in enumerate(beta_range):
    if i % 10 == 0:
        print(f"  {i+1}/{len(beta_range)}...", end="\r")
    
    # Use exact computation with small field to break symmetry
    _, _, _, angle = cw.implied_alpha_from_constraints(beta, J, h_sym, n)
    m = cw.exact_expectation_magnetisation(beta, J, h_sym, n)
    grad_I = cw.exact_gradient_multi_info_wrt_m(beta, J, h_sym, n)
    
    angles.append(angle)
    magnetisations.append(m)
    grad_I_values.append(grad_I)


    

In [ ]:
big_wide_figsize = (10, 5)
big_figsize = (8, 8)

In [ ]:
# Panel 1: Angle vs temperature
fig, ax = plt.subplots(figsize=big_wide_figsize)
ax.plot(beta_range / beta_c, angles, 'b-', linewidth=2)
ax.axvline(1.0, color='r', linestyle='--', label='$T_c$')
ax.axhline(10, color='gray', linestyle=':', label='10° threshold')
ax.set_xlabel(r'$\beta / \beta_c$')
ax.set_ylabel(r'Angle between $\nabla h$ and $\nabla E$ (degrees)')
ax.set_title('Constraint Equivalence Breakdown')
ax.legend()
ax.grid(True, alpha=0.3)
plt.savefig('fig-critical-scaling_angle-vs-temperature.pdf', dpi=150, bbox_inches='tight')

In [ ]:
# Panel 2: Magnetisation vs temperature
fig, ax = plt.subplots(figsize=big_wide_figsize)
ax.plot(beta_range / beta_c, np.abs(magnetisations), 'g-', linewidth=2)
ax.axvline(1.0, color='r', linestyle='--', label=r'$\beta_c$')
ax.set_xlabel(r'$\beta / \beta_c$')
ax.set_ylabel('$|m|$')
ax.legend()
ax.grid(True, alpha=0.3)
plt.savefig('fig-critical-scaling_magnetisation-vs-temperature.pdf', dpi=150, bbox_inches='tight')

In [ ]:
# Panel 3: ∇I vs temperature (key prediction!)
fig, ax = plt.subplots(figsize=big_wide_figsize)
ax.plot(beta_range / beta_c, np.abs(grad_I_values), 'purple', linewidth=2)
ax.axvline(1.0, color='r', linestyle='--', label=r'$\beta_c$')
ax.axhline(0.1, color='gray', linestyle=':', label='Threshold')
ax.set_xlabel(r'$\beta / \beta_c$')
ax.set_ylabel(r'$|\nabla_m I|$')
ax.legend()
ax.grid(True, alpha=0.3)
plt.savefig('fig-critical-scaling_multiinformation-gradient-vs-temperature', dpi=150, bbox_inches='tight')

In [ ]:
# Panel 4: Scaling analysis near T_c
fig, ax = plt.subplots(figsize=big_wide_figsize)
# Focus on region near beta_c
mask = (beta_range / beta_c > 0.8) & (beta_range / beta_c < 1.5)
delta_beta = np.abs(beta_range[mask] - beta_c) / beta_c
cos_theta = np.cos(np.radians(np.array(angles)[mask]))

# Plot 1 - cos(θ) vs |beta - beta_c|
ax.loglog(delta_beta, 1 - cos_theta, 'ro', markersize=4, label='Data')

# Fit power law
valid = (delta_beta > 0.01) & np.isfinite(cos_theta)
if np.sum(valid) > 5:
    coeffs = np.polyfit(np.log(delta_beta[valid]), np.log((1 - cos_theta[valid]) + 1e-10), 1)
    alpha = coeffs[0]
    ax.loglog(delta_beta[valid], np.exp(coeffs[1]) * delta_beta[valid]**alpha, 'b--', 
             linewidth=4, label=f'$\\alpha = {alpha:.2f}$')
    print(f"Fitted exponent α = {alpha:.3f}")
    print(f"(Gaussian prediction: α = 2.0)")

ax.set_xlabel(r'$|\beta - \beta_c| / \beta_c$')
ax.set_ylabel('$1 - \\cos(\\theta)$')
#ax.set_title('Critical Scaling')
ax.legend()
ax.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.savefig('prediction1_critical_scaling.pdf', dpi=150, bbox_inches='tight')
    


**Critical scaling breakdown**

*Top left*: Angle between constraint gradients $\nabla(\sum_i h_i)$ and $\nabla E$ jumps from $\approx 0\degree$ (Gaussian regime, $\beta < \beta_c$) to $\approx 65\degree$ (ordered phase, $\beta > \beta_c$). 
*Top right*: Order parameter $|m|$ transitions from 0 to $\approx 0.95$ at $\beta_c$. 
*Bottom left*: Multi-information gradient $|\nabla_m I|$ exhibits a sharp spike at $\beta_c$, directly demonstrating the theorem's condition $\nabla I \approx 0$ breaks down. 
*Bottom right*: Critical scaling analysis shows $1 - \cos(\theta) \sim |\beta - \beta_c|^{\alpha}$ with fitted exponent $\alpha = ??$, revealing non-Gaussian critical behavior.


**Key Observations**

* The angle between constraint gradients remains near $0\degree$ for $\beta < \beta_c$ (Gaussian regime), confirming $\nabla(\sum_i h_i) \approx \nabla H$ when $\nabla I \approx 0$.

* At the critical point $\beta = \beta_c$, the multi-information gradient $|\nabla_m I|$ spikes from $\approx 3$ to $\approx 20$, a 7-fold increase marking the breakdown of the theorem's condition.

* Above $\beta_c$, the angle saturates at $\approx 65 \degree$, showing the constraints imply fundamentally different natural parameters in the ordered phase.

* The critical scaling exponent $\alpha = -4.58$ deviates significantly from the Gaussian prediction ($\alpha = 2.0$), reflecting the singular nature of the phase transition.


# Experiment 2: Constraint Manifold Separation}

Next we illustrate how the energy and marginal entropy constraint manifolds evolve across different temperature regimes.





In [ ]:
beta_c = 1.0/J
T_c = 1/beta_c

beta_range = np.array([2.0 * beta_c, 1.2 * beta_c, beta_c, 0.8 * beta_c, 0.5 * beta_c])

# Panel 1: Energy-Entropy plane
fig, ax = plt.subplots(figsize=big_figsize)

for beta in beta_range:
    T = 1.0 / beta
    
    # Scan external field to trace out manifold
    h_range = np.linspace(-2.0, 2.0, 50)
    energies = []
    entropies = []
    
    for h in h_range:
        m = cw.curie_weiss_magnetisation(beta, J, h)
        if np.isnan(m):
            continue
        
        E = -J * m**2 / 2 - h * m
        S = cw.marginal_entropy(m, n)
        
        energies.append(E)
        entropies.append(S)
    
    label = fr'$\beta/\beta_c = {beta/beta_c:.2f}$'
    ax.plot(energies, entropies, 'o-', markersize=3, label=label, alpha=0.7)

ax.set_xlabel(r'Energy $\langle E \rangle$')
ax.set_ylabel(r'Marginal Entropy $h(m)$')
ax.legend()
ax.grid(True, alpha=0.3)
plt.savefig('fig-manifold-separation_different-phases.pdf', dpi=150, bbox_inches='tight')

**Constraint manifold separation** 

Trajectories in the $(E, \sum_i h_i)$ plane traced by varying external field $h$ at fixed temperatures. At low $\beta$ (blue), the relationship is nearly linear. At high $\beta$ (purple), strong curvature emerges, indicating different constraint manifolds. 



In [ ]:
# Panel 2: Separation vs magnetisation (now using exact computation)
fig, ax = plt.subplots(figsize=big_figsize)

# Sweep over external field h to get different magnetizations
h_range = np.linspace(0.01, 0.8, 50)  # Reduced points for exact computation

for beta in beta_range:
    angles = []
    m_vals = []
    
    for h_val in h_range:
        m = cw.exact_expectation_magnetisation(beta, J, h_val, n)
        _, _, _, angle = cw.implied_alpha_from_constraints(beta, J, h_val, n)
        angles.append(angle)
        m_vals.append(abs(m))
    
    label = rf'$\beta/\beta_c = {beta/beta_c:.2f}$'
    ax.plot(m_vals, angles, '-', linewidth=2, label=label, alpha=0.7)

ax.axhline(10, color='gray', linestyle=':', label='10° threshold')
ax.set_xlabel('Magnetisation $|m|$')
ax.set_ylabel('Angle (degrees)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.savefig('fig-manifold-separation_alignment-vs-order.pdf', dpi=150, bbox_inches='tight')

Angle between constraint gradients as a function of magnetization $|m|$ at different temperatures. At $\beta/\beta_c = 0.5$ (blue), the angle stays below $10°$ for all $|m|$. At $\beta/\beta_c = 2.0$ (purple), the angle exceeds $50°$, demonstrating constraint inequivalence.



**Key observations**

* In the Gaussian regime ($\beta/\beta_c = 0.5$, blue curves), the energy-entropy relationship is approximately linear and the angle remains small ($< 20°$) even as magnetization varies.

* At the critical point ($\beta/\beta_c = 1.0$, green), the curves begin to show curvature and the angle approaches $\approx 40°$.

* In the ordered phase ($\beta/\beta_c = 2.0$, purple), strong nonlinearity emerges and the angle exceeds $50°$, confirming the constraints have fundamentally separated.

*  The lower panel shows that for any fixed temperature, increasing $|m|$ (moving away from the weakly correlated regime) systematically increases the angle, consistent with growing correlations.

## Experiment 3: Regime Classification

Next we provide a comprehensive map of the energy-entropy equivalence across the full parameter space of magnetisation and temperature.



In [ ]:
beta_c = 1.0/J
beta_range = np.linspace(0.1*beta_c, 2.0*beta_c, 100) 

results = []
print(f"Computing regime classification with exact computation (n={n})...")
for i, beta in enumerate(beta_range):
    if i % 10 == 0:
        print(f"  {i+1}/{len(beta_range)}...", end="\r")
    
    # Use exact computation with small field
    _, _, _, angle = cw.implied_alpha_from_constraints(beta, J, h_sym, n)
    m = cw.exact_expectation_magnetisation(beta, J, h_sym, n)
    grad_I = cw.exact_gradient_multi_info_wrt_m(beta, J, h_sym, n)
    
    # Classify regime
    if abs(m) < 0.3:
        regime = 'Gaussian)'
    elif abs(m) > 0.6:
        regime = 'Ordered'
    else:
        regime = 'Transition'
    
    results.append({
        'T': 1.0 / beta,
        'beta': beta,
        'm': abs(m),
        'angle': angle,
        'grad_I': abs(grad_I),
        'regime': regime
    })

print("\nDone!")

In [ ]:
# Create regime plot
fig, ax = plt.subplots(figsize=big_wide_figsize)

for result in results:
    if result['regime'] == 'Gaussian':
        color = 'green'
        marker = 'o'
        alpha = 0.7
    elif result['regime'] == 'Ordered':
        color = 'red'
        marker = 's'
        alpha = 0.7
    else:
        color = 'orange'
        marker = '^'
        alpha = 0.5
    
    ax.scatter(result['m'], result['angle'], c=color, marker=marker, 
              s=50, alpha=alpha, edgecolors='black', linewidths=0.5)

# Add regime boundaries
ax.axvline(0.3, color='gray', linestyle='--', linewidth=2, label='$|m| = 0.3$ (Gaussian boundary)')
ax.axvline(0.6, color='gray', linestyle='--', linewidth=2, label='$|m| = 0.6$ (Ordered boundary)')
ax.axhline(5, color='gray', linestyle=':', linewidth=2, label='$5°$ (angle threshold)')

# Legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='green', edgecolor='black', label='Gaussian'),
    Patch(facecolor='orange', edgecolor='black', label='Transition'),
    Patch(facecolor='red', edgecolor='black', label='Ordered')
]
ax.legend(handles=legend_elements, loc='upper left')

ax.set_xlabel('Magnetisation $|m|$')
ax.set_ylabel('Angle (degrees)')
ax.grid(True, alpha=0.3)
ax.set_xlim(-0.05, 1.0)
ax.set_ylim(-5, 20)

plt.tight_layout()
plt.savefig('fig-equivalence_regimes.pdf', dpi=150, bbox_inches='tight')

# Print statistics
gaussian_count = sum(1 for r in results if r['regime'] == 'Gaussian')
ordered_count = sum(1 for r in results if r['regime'] == 'Ordered')
transition_count = sum(1 for r in results if r['regime'] == 'Transition')

print(f"\nRegime Statistics:")
print(f"  Gaussian (equivalence holds): {gaussian_count}/{len(results)} ({100*gaussian_count/len(results):.1f}%)")
print(f"  Transition: {transition_count}/{len(results)} ({100*transition_count/len(results):.1f}%)")
print(f"  Ordered (equivalence fails): {ordered_count}/{len(results)} ({100*ordered_count/len(results):.1f}%)")
print()

**Energy-entropy equivalence regimes.** Each point represents a state at temperature $\beta$ with magnetization $|m|$. *Green circles*: Gaussian regime ($|m| < 0.3$, angle $< 10°$) where equivalence holds. *Orange triangles*: Transition regime. *Red squares*: Ordered phase ($|m| > 0.6$ or angle $> 30°$) where equivalence fails. The vertical dashed line marks $|m| = 0.3$ (Gaussian boundary), and the horizontal dotted line marks the $10°$ angle threshold. Of 200 sampled states, 26\% satisfy the equivalence condition while 74\% show breakdown.}


**Key observations**

* A clear boundary emerges at $|m| \approx 0.3$: below this threshold, nearly all states exhibit angles $< 10°$ (green region), confirming equivalence.

* Above $|m| \approx 0.3$, the angle increases monotonically with magnetization, reaching $\approx 70°$ at $|m| \approx 1$ (fully ordered state).

* The transition is sharp: there are few intermediate points (orange triangles), suggesting the equivalence either holds or decisively fails with little ambiguity.

* The distribution is bimodal: 26\% in the Gaussian regime (equivalence), 74\% in the ordered phase (no equivalence), reflecting the natural division of thermodynamic states.


### Experiment: Alpha Alignment Across Phase Transition

Testing: Angle between α_energy and α_entropy across phase transition")

\subsection{Experiment 4: Implied Natural Parameters}

Figure~\ref{fig:implied-alpha} directly demonstrates that the angle between constraint gradients corresponds to the angle between the natural parameters $\alpha$ implied by each constraint.

\begin{figure}[htbp]
\centering
\includegraphics[width=\textwidth]{implied_alpha_angle.pdf}
\caption{\textbf{Angle between implied natural parameters.} 
\textit{Top row}: Evolution of implied $\alpha$ parameters (left), multi-information gradient $|\nabla_m I|$ (center), and angular misalignment (right) across the phase transition. The top-center panel directly shows the theorem's condition: $|\nabla_m I| \approx 3$ above $T_c$ (small) vs.\ spike to $\approx 20$ at $T_c$ (large).
\textit{Bottom row}: Trajectories in $(\theta_{\text{energy}}, \theta_{\text{marginal}})$ space obtained by varying external field $h$ at fixed temperatures, where $\theta = -\beta \alpha$ is the natural parameter. At high $T$ (left, green), the trajectory lies on the diagonal (perfect alignment). At $T_c$ (center, orange), slight deviation appears. At low $T$ (right, red), the trajectory shows a pronounced ``kink'' crossing the diagonal, demonstrating the constraints imply different natural parameters.}
\label{fig:implied-alpha}
\end{figure}

\textbf{Key observations:}
\begin{enumerate}
\item \textbf{Top-left panel}: The three curves ($\alpha_{\text{energy}}$ from $\nabla E$, $\alpha_H$ from $\nabla H$, and $\alpha_{\text{marginal}}$ from $\nabla(\sum_i h_i)$) diverge dramatically below $T_c$, showing the constraints imply different natural parameters in the ordered phase.

\item \textbf{Top-center panel}: The multi-information gradient $|\nabla_m I|$ provides the direct diagnostic for equivalence breakdown. Values $\approx 3$ above $T_c$ grow to $\approx 20$ at $T_c$, a clear quantitative threshold.

\item \textbf{Top-right panel}: The angle follows $|\nabla_m I|$ closely, jumping from $\approx 0°$ to $\approx 45°$ at $T_c$ and saturating at $\approx 65°$ in the ordered phase.

\item \textbf{Bottom-left panel (Gaussian regime)}: The green trajectory traces $h = -0.5 \to +0.5$ at $T/T_c = 2.0$. Points lie almost exactly on the diagonal ($\theta_{\text{energy}} = \theta_{\text{marginal}}$), confirming the constraints are equivalent when $\nabla I \approx 0$.

\item \textbf{Bottom-center panel (Critical point)}: At $T = T_c$ (orange), the trajectory still approximately follows the diagonal but with visible curvature, indicating the onset of inequivalence.

\item \textbf{Bottom-right panel (Ordered phase)}: At $T/T_c = 0.5$ (red), the trajectory exhibits a dramatic ``kink'' structure, crossing the diagonal and demonstrating that the natural parameters implied by the two constraints point in fundamentally different directions.
\end{enumerate}


In [ ]:
"""
Experiment: Implied Alpha Angle Between Constraints

Shows that the angle between constraint gradients is the angle between
the implied natural parameters (α) from energy vs marginal entropy constraints.
"""

import numpy as np
import matplotlib.pyplot as plt
import curie_weiss_equivalence as cw



In [ ]:
from curie_weiss_equivalence import implied_alpha_from_constraints

In [ ]:
J = 1.0
T_c = J
T_range = np.linspace(0.5 * T_c, 2.0 * T_c, 100)  # Reduced for exact computation
beta_range = 1.0 / T_range

results = []
print(f"Computing implied alpha analysis with exact computation (n={n})...")
for i, beta in enumerate(beta_range):
    if i % 10 == 0:
        print(f"  {i+1}/{len(beta_range)}...", end="\r")
    
    # Use exact computation with small field
    alpha_E, alpha_H, alpha_marginal, angle = cw.implied_alpha_from_constraints(beta, J, h_sym, n)
    m = cw.exact_expectation_magnetisation(beta, J, h_sym, n)
    grad_I = cw.exact_gradient_multi_info_wrt_m(beta, J, h_sym, n)
    
    results.append({
        'beta': beta,
        'm': abs(m),
        'alpha_energy': alpha_E,
        'alpha_from_H': alpha_H,
        'alpha_marginal': alpha_marginal,
        'angle': angle,
        'grad_I': abs(grad_I)
    })

print("\nDone!")



In [ ]:
# Panel 1: Implied alphas vs temperature
fig, ax = plt.subplots(figsize=big_figsize)
beta_vals = [r['beta'] for r in results]
ax.plot(np.array(beta_vals) / beta_c, [r['alpha_energy'] for r in results], 
        'b-', linewidth=2, label=r'$\alpha_{energy}$ (from $\nabla E$)')
ax.plot(np.array(beta_vals) / beta_c, [r['alpha_from_H'] for r in results], 
        'g--', linewidth=2, label=r'$\alpha_{H}$ (from $\nabla H$)')
ax.plot(np.array(beta_vals) / beta_c, [r['alpha_marginal'] for r in results], 
        'r:', linewidth=2, label=r'$\alpha_{marginal}$ (from $\nabla \Sigma h_i$)')
ax.axvline(1.0, color='gray', linestyle='--', alpha=0.5, label=r'$\beta_c$')
ax.set_xlabel(r'$\beta / \beta_c$')
ax.set_ylabel(r'Implied $\alpha$ (natural parameter)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.savefig('fig-implied-alpha_vs-temperature.pdf', dpi=150, bbox_inches='tight')

In [ ]:
# Panel 2: |∇I| - the theorem condition
fig, ax = plt.subplots(figsize=big_figsize)
grad_I_vals = [r['grad_I'] for r in results]
ax.plot(np.array(beta_vals) / beta_c, grad_I_vals, 'purple', linewidth=4)
ax.axvline(1.0, color='gray', linestyle='--', alpha=0.5, label='$T_c$', linewidth=2)
ax.axhline(0.1, color='red', linestyle=':', alpha=0.5, label='Threshold', linewidth=2)
ax.set_xlabel(r'$\beta / \beta_c$')
ax.set_ylabel(r'$|\nabla_m I|$ (multi-information gradient)')
ax.legend()
ax.grid(True, alpha=0.3)
#ax.set_yscale('log')
ax.set_xlim([0.4, 2.1])
plt.savefig('fig-implied-alpha_theorem-condition.pdf', dpi=150, bbox_inches='tight')

In [ ]:
# Panel 3: Angle vs temperature
fig, ax = plt.subplots(figsize=big_figsize)
angles = [r['angle'] for r in results]
ax.plot(np.array(beta_vals) / beta_c, angles, 'b-', linewidth=2)
ax.axvline(1.0, color='gray', linestyle='--', alpha=0.5, label=r'$\beta_c$')
ax.axhline(10, color='red', linestyle=':', alpha=0.5, label='10° threshold')
ax.set_xlabel(r'$\beta / \beta_c$')
ax.set_ylabel(r'Angle between implied $\alpha$s (degrees)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.savefig('fig-implied-alpha_angular-mismatch.pdf', dpi=150, bbox_inches='tight')

In [ ]:
# Panel 4: Alpha space trajectory (high T) - sweep h (now using exact computation)
fig, ax = plt.subplots(figsize=big_figsize)
beta_low = 0.5 * beta_c
h_range = np.linspace(-0.5, 0.5, 20)  # Reduced for exact computation
theta_E_traj = []
theta_M_traj = []
for h_val in h_range:
    # Use exact computation
    alpha_E, alpha_H, alpha_M, _ = cw.implied_alpha_from_constraints(beta_low, J, h_val, n)
    # Scale by beta to get natural parameters θ = -β*α
    theta_E_traj.append(-beta_low * alpha_E)
    theta_M_traj.append(-beta_low * alpha_M)

ax.plot(theta_E_traj, theta_M_traj, 'go-', linewidth=2, markersize=4, alpha=0.7, label='Trajectory')
# Add diagonal reference
if theta_E_traj:
    data_range = [min(theta_E_traj + theta_M_traj), max(theta_E_traj + theta_M_traj)]
    ax.plot(data_range, data_range, 'k--', linewidth=2, alpha=0.5, label='Perfect alignment')
ax.set_xlabel(r'$\theta_{energy} = -\beta \alpha_{energy}$')
ax.set_ylabel(r'$\theta_{marginal} = -\beta \alpha_{marginal}$')
#ax.set_title(fr'Low T ($\beta/\beta_c$ = 0.5): Gaussian Regime')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_aspect('equal', adjustable='box')
plt.savefig('fig-implied-alpha_low-beta.pdf', dpi=150, bbox_inches='tight')

In [ ]:
# Panel 5: Alpha space trajectory (near Tc) - sweep h (now using exact computation)
fig, ax = plt.subplots(figsize=big_figsize)
beta_crit = beta_c
h_range = np.linspace(-0.5, 0.5, 20)  # Reduced for exact computation
theta_E_traj = []
theta_M_traj = []
for h_val in h_range:
    # Use exact computation
    alpha_E, alpha_H, alpha_M, _ = cw.implied_alpha_from_constraints(beta_crit, J, h_val, n)
    # Scale by beta to get natural parameters θ = -β*α
    theta_E_traj.append(-beta_crit * alpha_E)
    theta_M_traj.append(-beta_crit * alpha_M)

ax.plot(theta_E_traj, theta_M_traj, 'o-', color='orange', linewidth=2, markersize=4, alpha=0.7, label='Trajectory')
# Add diagonal reference
if theta_E_traj:
    data_range = [min(theta_E_traj + theta_M_traj), max(theta_E_traj + theta_M_traj)]
    ax.plot(data_range, data_range, 'k--', linewidth=1, alpha=0.5, label='Perfect alignment')
ax.set_xlabel(r'$\theta_{energy} = -\beta \alpha_{energy}$')
ax.set_ylabel(r'$\theta_{marginal} = -\beta \alpha_{marginal}$')
#ax.set_title(fr'At $\beta_c$ ($\beta/\beta_c = 1.0$): Critical Point')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_aspect('equal', adjustable='box')
plt.savefig('fig-implied-alpha_critical-beta.pdf', dpi=150, bbox_inches='tight')

In [ ]:
# Panel 6: Alpha space trajectory (low T) - sweep h (now using exact computation)
fig, ax = plt.subplots(figsize=big_figsize)
beta_high = 2.0 * beta_c
h_range = np.linspace(-0.5, 0.5, 20)  # Reduced for exact computation
theta_E_traj = []
theta_M_traj = []
for h_val in h_range:
    # Use exact computation
    alpha_E, alpha_H, alpha_M, _ = cw.implied_alpha_from_constraints(beta_high, J, h_val, n)
    # Scale by beta to get natural parameters θ = -β*α
    theta_E_traj.append(-beta_high * alpha_E)
    theta_M_traj.append(-beta_high * alpha_M)

ax.plot(theta_E_traj, theta_M_traj, 'rs-', linewidth=2, markersize=4, alpha=0.7, label='Trajectory')
# Add diagonal reference
if theta_E_traj:
    data_range = [min(theta_E_traj + theta_M_traj), max(theta_E_traj + theta_M_traj)]
    ax.plot(data_range, data_range, 'k--', linewidth=1, alpha=0.5, label='Perfect alignment')
ax.set_xlabel(r'$\theta_{energy} = -\beta \alpha_{energy}$')
ax.set_ylabel(r'$\theta_{marginal} = -\beta \alpha_{marginal}$')
#ax.set_title(rf'High $\beta$ ($\beta/\beta_c = 2.0$): Ordered Phase')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.savefig('fig-implied-alpha_high-beta.pdf', dpi=150, bbox_inches='tight')

### Summary Statistics


In [ ]:

gaussian_results = [r for r in results if r['m'] < 0.3]
ordered_results = [r for r in results if r['m'] > 0.6]

print("Gaussian Regime (|m| < 0.3):")
if gaussian_results:
    avg_angle = np.mean([r['angle'] for r in gaussian_results])
    max_diff = np.max([abs(r['alpha_energy'] - r['alpha_marginal']) for r in gaussian_results])
    print(f"  Average angle: {avg_angle:.2f}°")
    print(f"  Max |α_energy - α_marginal|: {max_diff:.4f}")
    print(f"  ✓ Implied alphas are ALIGNED")
print()

print("Ordered Phase (|m| > 0.6):")
if ordered_results:
    avg_angle = np.mean([r['angle'] for r in ordered_results])
    min_diff = np.min([abs(r['alpha_energy'] - r['alpha_marginal']) for r in ordered_results])
    print(f"  Average angle: {avg_angle:.2f}°")
    print(f"  Min |α_energy - α_marginal|: {min_diff:.4f}")
    print(f"  ✓ Implied alphas are MISALIGNED")
print()

print("Interpretation:")
print("  - The angle between constraint gradients IS the angle between")
print("    the natural parameters (α) implied by each constraint")
print("  - Energy constraint directly specifies α_energy")
print("  - Marginal entropy implies α_marginal = α_energy + correction from ∇I")
print("  - When ∇I ≈ 0 (Gaussian): constraints agree on α → equivalence")
print("  - When ∇I large (Ordered): constraints imply different α → no equivalence")
print()
print("="*70)

## Summary and Implications

These experiments provide validation of our energy-entropy equivalence theorem.

* **Direct verification**: The multi-information gradient $|\nabla_m I|$ serves as a quantitative diagnostic. Values $\lesssim 5$ indicate equivalence; values $\gtrsim 10$ indicate breakdown.

* **Phase transition signature**: The equivalence breaks down precisely at the phase transition, where long-range correlations emerge. This connects the information-geometric condition $\nabla I \approx 0$ to thermodynamic phase structure.

* **Natural parameter interpretation**: The ``angle'' between constraint gradients is literally the angle between the natural parameters $\alpha$ implied by each constraint. When $\nabla I \approx 0$, both constraints imply the same $\alpha$ (energy conservation is equivalent to marginal entropy conservation). When $\nabla I \gg 0$, they imply different $\alpha$'s (the constraints are physically distinct).

* **Practical threshold**: The boundary $|m| \approx 0.3$ and angle $\approx 10°$ provide concrete criteria for when phenomenological energy models can safely replace information-theoretic constraints.
\end{enumerate**

The Curie-Weiss model thus demonstrates that energy-entropy equivalence is not a universal property but a regime-dependent phenomenon, holding precisely when correlations (measured by $\nabla I$) remain weak enough to preserve the alignment of natural parameters across different constraint manifolds.

